# Evaluate Performance of Baseline Models
__Model performance will be evaluated on:__
1. In-Distribution sample (SNLI test split), in zero-shot settings (see [Part 1](https://github.com/shashiniyer/adversarial_nli_gpt2/blob/main/gpt2-medium/notebooks_and_scripts/Evaluation_Baselines_Part1.ipynb))
2. The following Out-of-Distribution samples:
    - HANS dataset (validation split), in zero-shot settings  (see [Part 1](https://github.com/shashiniyer/adversarial_nli_gpt2/blob/main/gpt2-medium/notebooks_and_scripts/Evaluation_Baselines_Part1.ipynb))
    - NLI Diagnostics dataset, in zero-shot settings  (see [Part 1](https://github.com/shashiniyer/adversarial_nli_gpt2/blob/main/gpt2-medium/notebooks_and_scripts/Evaluation_Baselines_Part1.ipynb))
    - Stress Test datasets, in zero-shot settings  (see [Part 1](https://github.com/shashiniyer/adversarial_nli_gpt2/blob/main/gpt2-medium/notebooks_and_scripts/Evaluation_Baselines_Part1.ipynb))
    - ANLI datasets (test splits), after fine-tuning the model for each round (_this notebook_)

__Performance indicators:__ Classification accuracy and $R_K$


## 1. Imports and Global Settings

In [1]:
from datasets import load_dataset, disable_caching
from transformers import GPT2TokenizerFast, DataCollatorWithPadding, set_seed
import torch
from torch.nn.functional import one_hot
import sys
sys.path.append('..')
from utils_ import tokenize, train_classifier, evaluate_acc_rk
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
set_seed(42)
#disable_caching()

In [2]:
# set up tokeniser
# padding to left because GPT2 uses last token for prediction
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2-medium", padding_side = 'left', \
                                              padding = True, truncation = True)
tokenizer.pad_token = tokenizer.eos_token # pad with 'eos' token

In [3]:
# set up data collator - https://huggingface.co/docs/transformers/main_classes/data_collator
# this is a (callable) helper object that sends batches of data to the model
data_collator = DataCollatorWithPadding(tokenizer, padding = 'max_length', \
                                         return_tensors = 'pt', max_length = 128)

## 2. Out-of-Distribution Evaluation - ANLI - Fine-Tuning
### 2.1. Data Read + Pre-Processing
- Get ANLI Datasets
- One-hot encode labels
- Tokenise data

In [4]:
# function to read in data and pre-process
def anli_data(split, tokenizer):

    # read in data
    data = load_dataset('anli', split = split)
    
    # one-hot encode labels
    data = data.map(lambda x: {'label': one_hot(torch.tensor(x['label']), 3).type(torch.float32).numpy()}, \
        batched = True)
    
    # tokenize data
    data = data.map(lambda x: tokenize(tokenizer, x['premise'] + '|' + x['hypothesis']))
    len_bef_exclusion = len(data)

    # exclude instances with > 128 tokens
    data = data.filter(lambda x: x['exclude'] == False)
    len_aft_exclusion = len(data)

    # print message if instances were in fact excluded
    if len_bef_exclusion - len_aft_exclusion > 0:

        print(f'Split: {split} - {len_bef_exclusion - len_aft_exclusion} ' + \
              f'({(len_bef_exclusion/len_aft_exclusion - 1)*100:>2f}%) sequences excluded')
    
    # keep only needed columns, set data format to PyTorch
    data.set_format(type = 'torch', columns = ['label', 'input_ids', 'attention_mask'])
    
    # store in data_dict
    return(data)

### 2.2. Fine-tune both Baseline models using corresponding training data for each ANLI round, then evaluate

In [5]:
# set up dictionary of the models/file_names
models = {'Unfiltered': 'baseline_unfiltered.pth', 'Random 190k Subset': 'baseline_random_190k.pth'}

In [6]:
for model_name, file_name in models.items(): # two baseline models
    
    for rd in ['r1', 'r2', 'r3']: # rounds
        
        # fine-tune
        tr = anli_data('train_' + rd, tokenizer)
        tr_model = torch.load(file_name)
        tr_dataloader = torch.utils.data.DataLoader(tr, batch_size=32, shuffle=True, collate_fn=data_collator)
        optimizer = torch.optim.Adam(tr_model.parameters(), lr = 1e-5)
        trained_classifier = train_classifier(tr_model, tr_dataloader, optimizer, device)
        del tr
        
        # evaluate
        te = anli_data('test_' + rd, tokenizer)
        te_dataloader = torch.utils.data.DataLoader(te, batch_size=32, collate_fn=data_collator)
        acc, rk = evaluate_acc_rk(trained_classifier, te_dataloader, device)
        del te
        del tr_model
        print(f'Round: {rd} - Model: {model_name} - Accuracy: {acc*100:>3f}%, RK: {rk:>3f}')       

Reusing dataset anli (/home/shana92/.cache/huggingface/datasets/anli/plain_text/0.1.0/aabce88453b06dff21c201855ea83283bab0390bff746deadb30b65695755c0b)
Parameter 'function'=<function anli_data.<locals>.<lambda> at 0x7f49d0c71ee0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
Loading cached processed dataset at /home/shana92/.cache/huggingface/datasets/anli/plain_text/0.1.0/aabce88453b06dff21c201855ea83283bab0390bff746deadb30b65695755c0b/cache-1c80317fa3b1799d.arrow
Loading cached processed dataset at /home/shana92/.cache/huggingface/datasets/anli/plain_text/0.1.0/aabce88453b0

Split: train_r1 - 446 (2.703030%) sequences excluded


  0%|          | 0/516 [00:00<?, ?it/s]

loss: 0.572932  [    0/16500]
loss: 0.629478  [ 1632/16500]
loss: 0.664723  [ 3264/16500]
loss: 0.644225  [ 4896/16500]
loss: 0.627973  [ 6528/16500]
loss: 0.605726  [ 8160/16500]
loss: 0.634016  [ 9792/16500]
loss: 0.599893  [11424/16500]
loss: 0.601813  [13056/16500]
loss: 0.639973  [14688/16500]
loss: 0.623417  [16320/16500]
Epoch average loss: 0.6221457123756409


  0%|          | 0/516 [00:00<?, ?it/s]

...

loss: 0.405011  [    0/92991]
loss: 0.459857  [ 9280/92991]
loss: 0.437436  [18560/92991]
loss: 0.539131  [27840/92991]
loss: 0.445319  [37120/92991]
loss: 0.519733  [46400/92991]
loss: 0.452904  [55680/92991]
loss: 0.394394  [64960/92991]
loss: 0.348360  [74240/92991]
loss: 0.378628  [83520/92991]
loss: 0.419476  [92800/92991]
Epoch average loss: 0.454674631357193
Done!


Reusing dataset anli (/home/shana92/.cache/huggingface/datasets/anli/plain_text/0.1.0/aabce88453b06dff21c201855ea83283bab0390bff746deadb30b65695755c0b)
Loading cached processed dataset at /home/shana92/.cache/huggingface/datasets/anli/plain_text/0.1.0/aabce88453b06dff21c201855ea83283bab0390bff746deadb30b65695755c0b/cache-47229389571aa876.arrow
Loading cached processed dataset at /home/shana92/.cache/huggingface/datasets/anli/plain_text/0.1.0/aabce88453b06dff21c201855ea83283bab0390bff746deadb30b65695755c0b/cache-371ecd7b27cd8130.arrow
Loading cached processed dataset at /home/shana92/.cache/huggingface/datasets/anli/plain_text/0.1.0/aabce88453b06dff21c201855ea83283bab0390bff746deadb30b65695755c0b/cache-38aece53e1b59f38.arrow


Split: test_r3 - 90 (8.108108%) sequences excluded


  0%|          | 0/35 [00:00<?, ?it/s]

Round: r3 - Model: Random 190k Subset - Accuracy: 34.774774%, RK: 0.021023
